In [3]:
import re
import time
import zipfile
from pathlib import Path
from typing import Dict, List, Tuple, Optional

import requests
import pandas as pd
from bs4 import BeautifulSoup

# =========================
# CONFIG
# =========================
USER_AGENT = "Name (your.email@domain.com) - 13F overlap research"  # must include '@'
MIN_SECONDS_BETWEEN_REQUESTS = 0.22

BASE_OUTPUT_DIR = Path(r"C:\Users\reigh\Desktop\Fin Statements")
OUT_FOLDER_NAME = "13F_Bulk_Overlap"

# How many SEC bulk dataset ZIPs to auto-download (2 = last 2 filing cycles)
N_LATEST_DATASETS_TO_FETCH = 2

# Overlap behavior
DISTINGUISH_OPTIONS = False          # if True: overlap key = CUSIP|PUTCALL (usually not recommended)
EXCLUDE_OPTIONS_FROM_OVERLAP = True  # ignore rows with PUTCALL for overlap + weights/totals

# Overlap definition (per period)
OVERLAP_MIN_MANAGERS = 2

# "Consensus new positions" definition
NEW_CONSENSUS_MIN_MANAGERS = 2  # e.g., NEW in >=2 managers in the latest period

# Change classification threshold (avoid tiny noise)
WEIGHT_CHANGE_PCT_THRESHOLD = 0.05  # 0.05% weight move treated as "unchanged" band

SEC_13F_DATASETS_PAGE = "https://www.sec.gov/data-research/sec-markets-data/form-13f-data-sets"

EXCEL_MAX_ROWS = 1_000_000


# =========================
# SEC CLIENT
# =========================
class SECClient:
    def __init__(self, user_agent: str, min_interval_s: float = 0.22):
        if not user_agent or "@" not in user_agent:
            raise ValueError("Set USER_AGENT like 'Name (your@email.com) - purpose'.")
        self.s = requests.Session()
        self.s.headers.update({
            "User-Agent": user_agent,
            "Accept-Encoding": "gzip, deflate",
        })
        self.min_interval_s = min_interval_s
        self._last_request_ts = 0.0

    def _sleep_if_needed(self):
        now = time.time()
        dt = now - self._last_request_ts
        if dt < self.min_interval_s:
            time.sleep(self.min_interval_s - dt)

    def get(self, url: str, stream: bool = False, max_retries: int = 6) -> requests.Response:
        for attempt in range(max_retries):
            self._sleep_if_needed()
            r = self.s.get(url, timeout=60, stream=stream)
            self._last_request_ts = time.time()
            if r.status_code == 200:
                return r
            if r.status_code in (429, 500, 502, 503, 504):
                time.sleep(min(2 ** attempt, 16))
                continue
            r.raise_for_status()
        raise RuntimeError(f"Failed GET: {url}")


# =========================
# HELPERS
# =========================
def normalize_cik(x: str) -> str:
    s = re.sub(r"\D", "", x.strip())
    if not s:
        raise ValueError(f"Bad CIK: {x}")
    return str(int(s)).zfill(10)

def parse_dd_mon_yyyy(s: str) -> pd.Timestamp:
    return pd.to_datetime(s, format="%d-%b-%Y", errors="coerce")

def holding_key(df: pd.DataFrame) -> pd.Series:
    if DISTINGUISH_OPTIONS:
        return df["CUSIP"].astype(str) + "|" + df["PUTCALL"].fillna("").astype(str)
    return df["CUSIP"].astype(str)

def clean_sheet_name(s: str) -> str:
    s = re.sub(r"[^A-Za-z0-9 _-]", "", str(s)).strip()
    return (s[:31] or "sheet")

def write_df_split(writer: pd.ExcelWriter, df: pd.DataFrame, sheet_base: str, max_rows: int = EXCEL_MAX_ROWS) -> None:
    sheet_base = clean_sheet_name(sheet_base)
    if df is None or df.empty:
        pd.DataFrame().to_excel(writer, sheet_name=sheet_base[:31], index=False)
        return

    n = len(df)
    if n <= max_rows:
        df.to_excel(writer, sheet_name=sheet_base[:31], index=False)
        return

    k = 0
    start = 0
    while start < n:
        k += 1
        chunk = df.iloc[start:start + max_rows].copy()
        suffix = f"_{k}"
        name = (sheet_base[: (31 - len(suffix))] + suffix)[:31]
        chunk.to_excel(writer, sheet_name=name, index=False)
        start += max_rows

def safe_excel_writer_path(out_dir: Path, base_name: str) -> Path:
    """
    If the Excel file is open, write to a timestamped filename instead of crashing.
    """
    out_dir.mkdir(parents=True, exist_ok=True)
    base_path = out_dir / f"{base_name}.xlsx"
    try:
        with open(base_path, "ab"):
            pass
        return base_path
    except PermissionError:
        ts = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
        return out_dir / f"{base_name}_{ts}.xlsx"

def find_file_by_token(folder: Path, token: str) -> Path:
    token_u = token.upper()
    for p in folder.rglob("*"):
        if p.is_file() and token_u in p.name.upper() and p.suffix.lower() in {".tsv", ".txt"}:
            return p
    raise FileNotFoundError(f"Could not find a TSV/TXT file containing '{token}' under {folder}")

def compute_pairwise(keys_by_mgr: Dict[str, set]) -> pd.DataFrame:
    mgrs = sorted(keys_by_mgr.keys())
    rows = []
    for i in range(len(mgrs)):
        for j in range(i + 1, len(mgrs)):
            a, b = mgrs[i], mgrs[j]
            sa, sb = keys_by_mgr[a], keys_by_mgr[b]
            inter = len(sa & sb)
            union = len(sa | sb)
            rows.append({
                "ManagerA": a,
                "ManagerB": b,
                "OverlapCount": inter,
                "UnionCount": union,
                "Jaccard": (inter / union) if union else 0.0
            })
    if not rows:
        return pd.DataFrame(columns=["ManagerA", "ManagerB", "OverlapCount", "UnionCount", "Jaccard"])
    return pd.DataFrame(rows).sort_values(["OverlapCount", "Jaccard"], ascending=False)

def manager_display_name(merged: pd.DataFrame) -> pd.Series:
    def f(r):
        nm = r.get("FILINGMANAGER_NAME", None)
        if isinstance(nm, str) and nm.strip():
            return nm.strip()
        return f"CIK{str(r['CIK']).zfill(10)}"
    return merged.apply(f, axis=1)


# =========================
# DATASET DISCOVERY + DOWNLOAD
# =========================
def list_dataset_links(sec: SECClient, max_items: int = 12) -> List[Tuple[str, str]]:
    html = sec.get(SEC_13F_DATASETS_PAGE).text
    soup = BeautifulSoup(html, "html.parser")
    links = []
    for a in soup.find_all("a"):
        href = a.get("href") or ""
        text = a.get_text(" ", strip=True)
        if href.endswith("_form13f.zip"):
            if href.startswith("/"):
                href = "https://www.sec.gov" + href
            links.append((text, href))
    # page is typically newest-to-oldest
    return links[:max_items]

def download_and_extract(sec: SECClient, zip_url: str, extract_dir: Path, zip_path: Path) -> None:
    extract_dir.mkdir(parents=True, exist_ok=True)
    zip_path.parent.mkdir(parents=True, exist_ok=True)

    # Download
    r = sec.get(zip_url, stream=True)
    with open(zip_path, "wb") as f:
        for chunk in r.iter_content(chunk_size=1024 * 1024):
            if chunk:
                f.write(chunk)

    # Extract
    with zipfile.ZipFile(zip_path, "r") as z:
        z.extractall(extract_dir)


# =========================
# LOAD + FILTER TABLES
# =========================
def load_submission(submission_path: Path) -> pd.DataFrame:
    df = pd.read_csv(submission_path, sep="\t", dtype=str)
    df["FILING_DATE_TS"] = df["FILING_DATE"].apply(parse_dd_mon_yyyy)
    df["PERIODOFREPORT_TS"] = df["PERIODOFREPORT"].apply(parse_dd_mon_yyyy)
    df["CIK"] = df["CIK"].astype(str).str.zfill(10)
    return df

def pick_latest_accessions_per_period(sub: pd.DataFrame, target_ciks: List[str]) -> pd.DataFrame:
    """
    One row per (CIK, PERIODOFREPORT) selecting latest FILING_DATE (so amendment wins).
    """
    sub = sub[sub["SUBMISSIONTYPE"].isin(["13F-HR", "13F-HR/A"])].copy()
    sub = sub[sub["CIK"].isin(target_ciks)].copy()
    sub = sub.sort_values(["CIK", "PERIODOFREPORT_TS", "FILING_DATE_TS"], ascending=[True, True, False])
    picked = sub.drop_duplicates(subset=["CIK", "PERIODOFREPORT_TS"], keep="first")
    return picked[[
        "ACCESSION_NUMBER", "CIK", "SUBMISSIONTYPE", "FILING_DATE", "PERIODOFREPORT",
        "FILING_DATE_TS", "PERIODOFREPORT_TS"
    ]].copy()

def load_coverpage(coverpage_path: Path, accessions: set) -> pd.DataFrame:
    df = pd.read_csv(coverpage_path, sep="\t", dtype=str)
    df = df[df["ACCESSION_NUMBER"].isin(accessions)].copy()
    return df

def load_infotable_chunked(infotable_path: Path, accessions: set) -> pd.DataFrame:
    usecols = [
        "ACCESSION_NUMBER", "INFOTABLE_SK", "NAMEOFISSUER", "TITLEOFCLASS",
        "CUSIP", "FIGI", "VALUE", "SSHPRNAMT", "SSHPRNAMTTYPE", "PUTCALL",
        "INVESTMENTDISCRETION", "OTHERMANAGER", "VOTING_AUTH_SOLE", "VOTING_AUTH_SHARED", "VOTING_AUTH_NONE"
    ]
    chunks = []
    for ch in pd.read_csv(infotable_path, sep="\t", dtype=str, usecols=lambda c: c in usecols, chunksize=400_000):
        ch = ch[ch["ACCESSION_NUMBER"].isin(accessions)]
        if not ch.empty:
            chunks.append(ch)
    if not chunks:
        return pd.DataFrame(columns=usecols)

    df = pd.concat(chunks, ignore_index=True)
    df["CUSIP"] = df["CUSIP"].astype(str).str.replace(r"\s+", "", regex=True)

    for c in ["VALUE", "SSHPRNAMT", "VOTING_AUTH_SOLE", "VOTING_AUTH_SHARED", "VOTING_AUTH_NONE"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c].astype(str).str.replace(",", ""), errors="coerce")

    if "PUTCALL" in df.columns:
        df["PUTCALL"] = df["PUTCALL"].replace("", pd.NA)

    return df


def process_one_dataset(extract_dir: Path, target_ciks: List[str]) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Returns:
      merged_rows: infotable rows joined with submission (+ coverpage if present)
      picked: picked submission rows (accessions selected) for transparency
    """
    submission_path = find_file_by_token(extract_dir, "SUBMISSION")
    infotable_path = find_file_by_token(extract_dir, "INFOTABLE")
    coverpage_path = None
    try:
        coverpage_path = find_file_by_token(extract_dir, "COVERPAGE")
    except FileNotFoundError:
        pass

    sub = load_submission(submission_path)
    picked_pp = pick_latest_accessions_per_period(sub, target_ciks)
    if picked_pp.empty:
        return pd.DataFrame(), pd.DataFrame()

    accessions = set(picked_pp["ACCESSION_NUMBER"].tolist())

    cover = pd.DataFrame()
    if coverpage_path is not None:
        cover = load_coverpage(coverpage_path, accessions)

    info = load_infotable_chunked(infotable_path, accessions)
    if info.empty:
        return pd.DataFrame(), picked_pp

    merged = info.merge(
        picked_pp[["ACCESSION_NUMBER", "CIK", "SUBMISSIONTYPE", "FILING_DATE", "PERIODOFREPORT", "FILING_DATE_TS", "PERIODOFREPORT_TS"]],
        on="ACCESSION_NUMBER",
        how="left"
    )

    if not cover.empty and "FILINGMANAGER_NAME" in cover.columns:
        merged = merged.merge(
            cover[["ACCESSION_NUMBER", "FILINGMANAGER_NAME", "ISAMENDMENT", "AMENDMENTTYPE"]],
            on="ACCESSION_NUMBER",
            how="left"
        )
    else:
        merged["FILINGMANAGER_NAME"] = None
        merged["ISAMENDMENT"] = None
        merged["AMENDMENTTYPE"] = None

    merged["manager"] = manager_display_name(merged)

    return merged, picked_pp


# =========================
# CHANGE ANALYSIS
# =========================
def build_fund_changes(mgr_cusip_vals: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Compare latest period vs previous period for each manager based on PERIODOFREPORT_TS.
    """
    if mgr_cusip_vals.empty:
        return pd.DataFrame(), pd.DataFrame()

    periods_by_mgr = (
        mgr_cusip_vals[["manager", "PERIODOFREPORT_TS"]].drop_duplicates()
        .sort_values(["manager", "PERIODOFREPORT_TS"], ascending=[True, False])
    )
    mgr_to_two = (
        periods_by_mgr.groupby("manager", dropna=False)["PERIODOFREPORT_TS"]
        .apply(lambda s: list(s.head(2)))
        .to_dict()
    )

    rows = []
    for mgr, per_list in mgr_to_two.items():
        if len(per_list) < 2:
            continue
        p_curr_ts, p_prev_ts = per_list[0], per_list[1]

        curr = mgr_cusip_vals[(mgr_cusip_vals["manager"] == mgr) & (mgr_cusip_vals["PERIODOFREPORT_TS"] == p_curr_ts)].copy()
        prev = mgr_cusip_vals[(mgr_cusip_vals["manager"] == mgr) & (mgr_cusip_vals["PERIODOFREPORT_TS"] == p_prev_ts)].copy()

        j = curr.merge(prev, on=["manager", "CUSIP"], how="outer", suffixes=("_curr", "_prev"))

        j["PERIOD_CURR_TS"] = p_curr_ts
        j["PERIOD_PREV_TS"] = p_prev_ts

        for col in ["PERIODOFREPORT", "NAMEOFISSUER", "TITLEOFCLASS"]:
            j[col] = j.get(f"{col}_curr").combine_first(j.get(f"{col}_prev"))

        for col in ["PositionValue_AsFiled", "Total13FValue_AsFiled", "WeightPct"]:
            j[f"{col}_curr"] = j[f"{col}_curr"].fillna(0)
            j[f"{col}_prev"] = j[f"{col}_prev"].fillna(0)

        j["DeltaValue_AsFiled"] = j["PositionValue_AsFiled_curr"] - j["PositionValue_AsFiled_prev"]
        j["DeltaWeightPct"] = j["WeightPct_curr"] - j["WeightPct_prev"]

        def classify(r):
            had_prev = r["PositionValue_AsFiled_prev"] > 0
            has_curr = r["PositionValue_AsFiled_curr"] > 0
            if (not had_prev) and has_curr:
                return "NEW"
            if had_prev and (not has_curr):
                return "EXIT"
            if abs(r["DeltaWeightPct"]) < WEIGHT_CHANGE_PCT_THRESHOLD:
                return "UNCHANGED"
            return "INCREASE" if r["DeltaWeightPct"] > 0 else "DECREASE"

        j["ChangeType"] = j.apply(classify, axis=1)

        out_cols = [
            "manager",
            "PERIODOFREPORT", "CUSIP", "NAMEOFISSUER", "TITLEOFCLASS",
            "PERIOD_PREV_TS", "PERIOD_CURR_TS",
            "PositionValue_AsFiled_prev", "WeightPct_prev",
            "PositionValue_AsFiled_curr", "WeightPct_curr",
            "DeltaValue_AsFiled", "DeltaWeightPct",
            "ChangeType"
        ]
        rows.append(j[out_cols])

    fund_changes_long = pd.concat(rows, ignore_index=True) if rows else pd.DataFrame()
    if fund_changes_long.empty:
        return fund_changes_long, pd.DataFrame()

    fund_changes_long["PERIOD_PREV"] = fund_changes_long["PERIOD_PREV_TS"].dt.strftime("%d-%b-%Y")
    fund_changes_long["PERIOD_CURR"] = fund_changes_long["PERIOD_CURR_TS"].dt.strftime("%d-%b-%Y")

    summ = (
        fund_changes_long.groupby(["manager", "PERIOD_PREV", "PERIOD_CURR"], dropna=False)
        .agg(
            NewPositions=("ChangeType", lambda s: (s == "NEW").sum()),
            Exits=("ChangeType", lambda s: (s == "EXIT").sum()),
            Increases=("ChangeType", lambda s: (s == "INCREASE").sum()),
            Decreases=("ChangeType", lambda s: (s == "DECREASE").sum()),
            Unchanged=("ChangeType", lambda s: (s == "UNCHANGED").sum()),
            SumAbsDeltaWeightPct=("DeltaWeightPct", lambda x: x.abs().sum()),
        )
        .reset_index()
        .sort_values(["SumAbsDeltaWeightPct"], ascending=[False])
    )

    fund_changes_long = fund_changes_long.sort_values(
        ["manager", "ChangeType", "DeltaWeightPct"],
        ascending=[True, True, False]
    )

    return fund_changes_long, summ


def build_new_positions_consensus(fund_changes_long: pd.DataFrame) -> pd.DataFrame:
    """
    Consensus NEW positions in the latest period: positions that are NEW for >= NEW_CONSENSUS_MIN_MANAGERS managers.
    """
    if fund_changes_long.empty:
        return pd.DataFrame()

    new_rows = fund_changes_long[fund_changes_long["ChangeType"] == "NEW"].copy()
    if new_rows.empty:
        return pd.DataFrame()

    # Identify latest "current period" across the dataset
    latest_curr_ts = new_rows["PERIOD_CURR_TS"].max()
    new_rows = new_rows[new_rows["PERIOD_CURR_TS"] == latest_curr_ts].copy()

    if new_rows.empty:
        return pd.DataFrame()

    g = new_rows.groupby(["PERIOD_CURR_TS", "PERIOD_CURR", "CUSIP", "NAMEOFISSUER", "TITLEOFCLASS"], dropna=False)

    consensus = g.agg(
        ManagersNew=("manager", "nunique"),
        TotalNewWeightPct=("WeightPct_curr", "sum"),
        AvgNewWeightPct=("WeightPct_curr", "mean"),
        MedianNewWeightPct=("WeightPct_curr", "median"),
        MaxNewWeightPct=("WeightPct_curr", "max"),
    ).reset_index()

    # Determine top holder among NEW adopters
    top_holder = (
        new_rows.sort_values(["CUSIP", "WeightPct_curr"], ascending=[True, False])
        .groupby(["CUSIP"], dropna=False)
        .head(1)[["CUSIP", "manager", "WeightPct_curr"]]
        .rename(columns={"manager": "TopNewHolder", "WeightPct_curr": "TopNewHolderWeightPct"})
    )
    consensus = consensus.merge(top_holder, on="CUSIP", how="left")

    # Manager list (small: 5–10 CIKs)
    mgr_list = (
        new_rows.groupby("CUSIP")["manager"]
        .apply(lambda s: ", ".join(sorted(set(map(str, s)))))
        .rename("ManagersNewList")
        .reset_index()
    )
    consensus = consensus.merge(mgr_list, on="CUSIP", how="left")

    # Filter to “consensus” threshold
    consensus = consensus[consensus["ManagersNew"] >= NEW_CONSENSUS_MIN_MANAGERS].copy()

    # Sort: strongest consensus first
    consensus = consensus.sort_values(
        ["ManagersNew", "TotalNewWeightPct", "MaxNewWeightPct"],
        ascending=[False, False, False]
    )

    return consensus


# =========================
# MAIN
# =========================
def interactive_bulk_13f():
    sec = SECClient(USER_AGENT, MIN_SECONDS_BETWEEN_REQUESTS)

    cik_input = input("Enter manager CIKs (comma-separated): ").strip()
    target_ciks = [normalize_cik(x) for x in cik_input.split(",") if x.strip()]
    if len(target_ciks) < 2:
        raise ValueError("Provide at least 2 CIKs.")

    print("\nDiscovering SEC 13F dataset links...")
    links = list_dataset_links(sec, max_items=10)
    if not links:
        raise RuntimeError("Could not find dataset links on the SEC page.")

    # Auto-select latest N datasets (no prompt)
    selected = links[:N_LATEST_DATASETS_TO_FETCH]
    print("\nAuto-selected datasets:")
    for i, (label, url) in enumerate(selected, start=1):
        print(f"  {i}) {label} | {url}")

    out_dir = BASE_OUTPUT_DIR / OUT_FOLDER_NAME
    datasets_dir = out_dir / "datasets"
    datasets_dir.mkdir(parents=True, exist_ok=True)

    merged_frames = []
    picked_frames = []

    for i, (label, url) in enumerate(selected, start=1):
        extract_dir = datasets_dir / f"extracted_{i:02d}"
        zip_path = datasets_dir / f"dataset_{i:02d}.zip"

        print(f"\nDownloading + extracting dataset {i}/{len(selected)}...")
        download_and_extract(sec, url, extract_dir, zip_path)

        merged_i, picked_i = process_one_dataset(extract_dir, target_ciks)
        if not picked_i.empty:
            picked_i = picked_i.copy()
            picked_i["DATASET_IDX"] = i
            picked_i["DATASET_LABEL"] = label
            picked_frames.append(picked_i)

        if not merged_i.empty:
            merged_i = merged_i.copy()
            merged_i["DATASET_IDX"] = i
            merged_i["DATASET_LABEL"] = label
            merged_frames.append(merged_i)

    if not merged_frames:
        raise RuntimeError("No holdings rows were found for your CIKs across the selected datasets.")

    merged = pd.concat(merged_frames, ignore_index=True)
    picked_all = pd.concat(picked_frames, ignore_index=True) if picked_frames else pd.DataFrame()

    # Base universe (option filter)
    base = merged.copy()
    if EXCLUDE_OPTIONS_FROM_OVERLAP and "PUTCALL" in base.columns:
        base = base[base["PUTCALL"].isna()]

    # Fund totals + weights (as filed)
    mgr_cusip_vals = (
        base.groupby(["PERIODOFREPORT", "PERIODOFREPORT_TS", "manager", "CUSIP"], dropna=False)["VALUE"]
        .sum(min_count=1)
        .rename("PositionValue_AsFiled")
        .reset_index()
    )

    fund_totals = (
        mgr_cusip_vals.groupby(["PERIODOFREPORT", "PERIODOFREPORT_TS", "manager"], dropna=False)["PositionValue_AsFiled"]
        .sum(min_count=1)
        .rename("Total13FValue_AsFiled")
        .reset_index()
    )

    mgr_cusip_vals = mgr_cusip_vals.merge(
        fund_totals, on=["PERIODOFREPORT", "PERIODOFREPORT_TS", "manager"], how="left"
    )
    mgr_cusip_vals["WeightPct"] = (mgr_cusip_vals["PositionValue_AsFiled"] / mgr_cusip_vals["Total13FValue_AsFiled"]) * 100

    labels = (
        base.groupby(["PERIODOFREPORT", "PERIODOFREPORT_TS", "CUSIP"], dropna=False)[["NAMEOFISSUER", "TITLEOFCLASS"]]
        .first()
        .reset_index()
    )
    mgr_cusip_vals = mgr_cusip_vals.merge(labels, on=["PERIODOFREPORT", "PERIODOFREPORT_TS", "CUSIP"], how="left")

    # Overlap counts per CUSIP per period
    cusip_mgr_counts = (
        mgr_cusip_vals.groupby(["PERIODOFREPORT", "PERIODOFREPORT_TS", "CUSIP"], dropna=False)["manager"]
        .nunique()
        .rename("ManagersHolding")
        .reset_index()
    )
    overlap_cusips = cusip_mgr_counts[cusip_mgr_counts["ManagersHolding"] >= OVERLAP_MIN_MANAGERS].copy()

    all_overlaps = (
        overlap_cusips.merge(
            mgr_cusip_vals.groupby(["PERIODOFREPORT", "PERIODOFREPORT_TS", "CUSIP"], dropna=False)["PositionValue_AsFiled"]
            .sum(min_count=1).rename("TotalValue_AsFiled_AllManagers").reset_index(),
            on=["PERIODOFREPORT", "PERIODOFREPORT_TS", "CUSIP"],
            how="left"
        )
        .merge(labels, on=["PERIODOFREPORT", "PERIODOFREPORT_TS", "CUSIP"], how="left")
        .sort_values(["PERIODOFREPORT_TS", "ManagersHolding", "TotalValue_AsFiled_AllManagers"],
                     ascending=[False, False, False])
    )

    overlap_weights_long = (
        mgr_cusip_vals.merge(overlap_cusips, on=["PERIODOFREPORT", "PERIODOFREPORT_TS", "CUSIP"], how="inner")
        .sort_values(["PERIODOFREPORT_TS", "ManagersHolding", "CUSIP", "WeightPct"],
                     ascending=[False, False, True, False])
    )

    overlap_weights_matrix = overlap_weights_long.pivot_table(
        index=["PERIODOFREPORT", "PERIODOFREPORT_TS", "CUSIP", "NAMEOFISSUER", "TITLEOFCLASS", "ManagersHolding"],
        columns="manager",
        values="WeightPct",
        aggfunc="first"
    ).reset_index()

    # Pairwise overlap per period
    pairwise_frames = []
    for (period, period_ts), dfp in base.groupby(["PERIODOFREPORT", "PERIODOFREPORT_TS"], dropna=False):
        keys_by_mgr = {}
        for mgr, dfm in dfp.groupby("manager"):
            keys_by_mgr[mgr] = set(holding_key(dfm).dropna().tolist())
        pw = compute_pairwise(keys_by_mgr)
        if not pw.empty:
            pw.insert(0, "PERIODOFREPORT", period)
            pw.insert(1, "PERIODOFREPORT_TS", period_ts)
            pairwise_frames.append(pw)
    pairwise_overlap = pd.concat(pairwise_frames, ignore_index=True) if pairwise_frames else pd.DataFrame()

    # Fund changes + consensus new
    fund_changes_long, fund_changes_summary = build_fund_changes(mgr_cusip_vals)
    new_positions_consensus = build_new_positions_consensus(fund_changes_long)

    # =========================
    # WRITE OUTPUT
    # =========================
    out_file = safe_excel_writer_path(out_dir, "13F_bulk_overlap_summary")

    with pd.ExcelWriter(out_file, engine="openpyxl") as writer:
        

        write_df_split(writer, pairwise_overlap, "pairwise_overlap")
        write_df_split(writer, all_overlaps, "all_overlaps")
        
        write_df_split(writer, overlap_weights_matrix, "overlap_weights_matrix")
        write_df_split(writer, overlap_weights_matrix, "overlap_weights_long")
        

        write_df_split(writer, fund_changes_long, "fund_changes_long")
        write_df_split(writer, fund_changes_summary, "fund_changes_summary")
        write_df_split(writer, new_positions_consensus, "new_positions_consensus")

        # Optional: per-manager holdings (sorted by period and weight)
        for mgr, dfm in mgr_cusip_vals.groupby("manager"):
            sheet = clean_sheet_name(mgr)
            dfm.sort_values(["PERIODOFREPORT_TS", "WeightPct"], ascending=[False, False]).to_excel(
                writer, sheet_name=sheet, index=False
            )

    print(f"\nSaved: {out_file}")
    print("\nNotes:")
    print(" - No dataset prompt anymore: the script auto-downloads the latest 2 SEC bulk 13F datasets.")
    print(" - If the output filename has a timestamp, your previous workbook was open in Excel.")
    print(" - new_positions_consensus shows positions that are NEW in the latest period for >= "
          f"{NEW_CONSENSUS_MIN_MANAGERS} managers.")


if __name__ == "__main__":
    interactive_bulk_13f()


ValueError: Provide at least 2 CIKs.